In [1]:
# HEATMAPS 

# Import the dependencies 
import pandas as pd
import gmaps 
import requests 

# Import the API key
from config import g_key

In [2]:
# Store CSV into DataFrame 

city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Clyde River,CA,2021-10-27 21:41:16,70.4692,-68.5914,32.23,93,75,0.00
1,1,Busselton,AU,2021-10-27 21:43:33,-33.6500,115.3333,45.05,75,83,9.33
2,2,Kapaa,US,2021-10-27 21:43:33,22.0752,-159.3190,82.38,77,20,5.01
3,3,Mataura,NZ,2021-10-27 21:43:33,-46.1927,168.8643,64.51,60,10,3.53
4,4,Arona,ES,2021-10-27 21:43:34,28.0996,-16.6810,64.71,64,20,13.80


In [3]:
# To use gmaps, data must be either an integer or floating point number 
city_data_df.dtypes

City ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
# CREATE A MAXIMUM TEMPERATURE HEATMAP 

# Configure gmaps to use API key 
gmaps.configure(api_key=g_key)

In [5]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]

temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [10]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]

# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=[max(temp, 0) for temp in max_temp], dissipating=False,
                                max_intensity = 300, point_radius=4)

# Center, zoom to show only 1 map, and add the heatmap layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Add the heat layer
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
# Heatmap of humidity 
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False,
                                max_intensity = 300, point_radius=4)

# Add the heat layer
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [12]:
# Heatmap of percent cloudiness 
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
cloudiness = city_data_df["Cloudiness"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=cloudiness, dissipating=False,
                                max_intensity = 300, point_radius=4)

# Add the heat layer
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# Heatmap of wind speed 

# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
wind_speed = city_data_df["Wind Speed"]

# Assign the figure variable.
fig = gmaps.figure(center=(30.0,31.0), zoom_level=1.5)

# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=wind_speed, dissipating=False,
                                max_intensity = 300, point_radius=4)

# Add the heat layer
fig.add_layer(heat_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [33]:
# Filter dataset to retried nw df dusplaying temps within range 

filter = ((city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)) 
preferred_cities_df = city_data_df.loc[filter]
preferred_cities_df.count()
preferred_cities_df.head()


,City ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Kapaa,US,2021-10-27 21:43:33,22.0752,-159.3190,82.38,77,20,5.01
6,6,Trincomalee,LK,2021-10-27 21:43:35,8.5711,81.2335,78.24,86,100,7.05
8,8,Laguna,US,2021-10-27 21:43:36,38.4210,-121.4238,75.15,65,1,3.00
12,12,Puerto Ayora,EC,2021-10-27 21:43:38,-0.7393,-90.3518,76.96,89,91,5.99
20,20,Victoria,HK,2021-10-27 21:43:32,22.2855,114.1577,77.38,82,82,1.99


In [35]:
# Make a copy of preferred DF 

hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Kapaa,US,82.38,22.0752,-159.3190,
6,Trincomalee,LK,78.24,8.5711,81.2335,
8,Laguna,US,75.15,38.4210,-121.4238,
12,Puerto Ayora,EC,76.96,-0.7393,-90.3518,
20,Victoria,HK,77.38,22.2855,114.1577,


In [38]:
# RETRIEVE HOTELS FROM A NEARBY SEARCH 


# Iterrate through hotel_df to get lats and lngs for params, then perform search
for index, row in hotel_df.iterrows():
    params = {
        "radius": 5000,
        "type": "lodging", 
        "key": g_key
        # Need to add location param --> use iteration below
    }
    
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Search time!
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    hotels = requests.get(base_url, params=params).json()
    
    # Grab first hotel from results and store the name 
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping")
    
    

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping


In [39]:
hotel_df.head()

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Kapaa,US,82.38,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
6,Trincomalee,LK,78.24,8.5711,81.2335,Welcombe Hotel
8,Laguna,US,75.15,38.4210,-121.4238,Holiday Inn Express & Suites Elk Grove Central...
12,Puerto Ayora,EC,76.96,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,Victoria,HK,77.38,22.2855,114.1577,Mini Hotel Central


In [42]:
# Add a heatmap of temps for vacation spots 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]


info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]



fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)

# Create marker layer 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info) 

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Show
fig

Figure(layout=FigureLayout(height='420px'))